In [2]:
import pandas as pd
import numpy as np


In [5]:
# Charger le DataFrame d'origine à partir du fichier CSV
df = pd.read_csv('add_anomly_3.csv')
df

C:\Users\MSI\AppData\Local\Temp\ipykernel_22196\963785144.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('add_anomly_3.csv')


,user_number,datetime,Consommation,anomalie,type
0,1,2016-01-03 01:19:43,0,0,NaN
1,1,2016-01-03 02:19:43,0,0,NaN
2,1,2016-01-03 03:19:43,0,0,NaN
3,1,2016-01-03 04:19:43,0,0,NaN
4,1,2016-01-03 05:39:54,0,0,NaN
...,...,...,...,...,...
422033,99999,2017-12-02 19:52:01,38,0,NaN
422034,99999,2017-12-02 20:52:01,67,0,NaN
422035,99999,2017-12-02 21:52:01,17,0,NaN
422036,99999,2017-12-02 22:52:01,4,0,NaN


In [6]:
# Paramètres
num_users = 20  # Nombre d'utilisateurs à sélectionner
user_range = range(1, 51)  # Plage des utilisateurs (1 à 50)
days_min = 4  # Nombre minimum de jours successifs
days_max = 8  # Nombre maximum de jours successifs
gap_min = 20  # Valeur minimum du gap
gap_max = 50  # Valeur maximum du gap

# Vérifier et convertir la colonne datetime au format datetime
df['datetime'] = pd.to_datetime(df['datetime'], errors='coerce', dayfirst=True)
df = df.dropna(subset=['datetime'])  # Supprimer les lignes avec des dates invalides si nécessaire

# Vérifier si la conversion a réussi
print("Aperçu de la colonne 'datetime' après conversion :")
print(df[['datetime']].head())  # Afficher uniquement la colonne datetime

# Vérification de NaT dans 'datetime'
na_dates = df['datetime'].isna().sum()
if na_dates > 0:
    print(f"Attention : {na_dates} valeurs sont NaT (problème de conversion de dates).")

# Sélectionner 20 utilisateurs aléatoires dans la plage [1, 50]
random_users = np.random.choice(user_range, size=num_users, replace=False)

# Créer une copie du DataFrame global pour éviter de modifier l'original
df_combined = df.copy()

# Pour chaque utilisateur sélectionné, appliquer l'anomalie
for user_id in random_users:
    # Sélectionner les données de l'utilisateur spécifique
    user_data = df[df['user_number'] == user_id].copy()

    # Créer une copie de user_data pour ajouter l'anomalie (user_data_anomaly)
    user_data_anomaly = user_data.copy()

    # Sélectionner un nombre aléatoire de jours successifs (entre 4 et 8 jours)
    num_days_to_anomaly = np.random.randint(days_min, days_max + 1)

    # Sélectionner une date aléatoire pour commencer l'anomalie
    random_start_date = np.random.choice(user_data_anomaly['datetime'].dt.date.unique(), size=1, replace=False)[0]

    # Appliquer l'anomalie sur les jours successifs à partir de la date choisie
    for day_offset in range(num_days_to_anomaly):  
        selected_date = pd.Timestamp(random_start_date) + pd.Timedelta(days=day_offset)  # Calculer la date successive

        # Appliquer l'anomalie sur toutes les lignes de ces jours successifs
        condition = user_data_anomaly['datetime'].dt.date == selected_date.date()  # Trouver les lignes correspondant à cette date
        gap_value = np.random.randint(gap_min, gap_max + 1)  # Générer un gap aléatoire entre 20 et 50
        user_data_anomaly.loc[condition, 'Consommation'] += gap_value  # Ajouter le gap de consommation
        user_data_anomaly.loc[condition, 'anomalie'] = 1  # Marquer comme anomalie
        user_data_anomaly.loc[condition, 'type'] = 'Fuite de consommation'  # Type d'anomalie

    # Générer un nouveau numéro d'utilisateur aléatoire entre 20000 et 30000 pour les données modifiées
    new_user_id = np.random.randint(20000, 30001)  # Numéro d'utilisateur aléatoire dans la plage [20000, 30000]
    user_data_anomaly['user_number'] = new_user_id  # Assigner le nouveau numéro d'utilisateur

    # Concaténer les données modifiées de l'utilisateur avec le DataFrame global
    df_combined = pd.concat([df_combined, user_data_anomaly])

# Enregistrer le DataFrame combiné dans un fichier CSV
df_combined.to_csv('add_anomly_20_users_with_new_ids.csv', index=False)

# Afficher un aperçu des données combinées, en particulier la colonne datetime
print("Aperçu des données combinées :")
print(df_combined[['datetime', 'user_number', 'Consommation', 'anomalie', 'type']].head())

Aperçu de la colonne 'datetime' après conversion :
             datetime
0 2016-03-01 01:19:43
1 2016-03-01 02:19:43
2 2016-03-01 03:19:43
3 2016-03-01 04:19:43
4 2016-03-01 05:39:54
Aperçu des données combinées :
             datetime  user_number  Consommation  anomalie type
0 2016-03-01 01:19:43            1             0         0  NaN
1 2016-03-01 02:19:43            1             0         0  NaN
2 2016-03-01 03:19:43            1             0         0  NaN
3 2016-03-01 04:19:43            1             0         0  NaN
4 2016-03-01 05:39:54            1             0         0  NaN


In [19]:
df_combined

,user_number,datetime,Consommation,anomalie,type
0,1,2016-03-01 01:19:43,0,0,NaN
1,1,2016-03-01 02:19:43,0,0,NaN
2,1,2016-03-01 03:19:43,0,0,NaN
3,1,2016-03-01 04:19:43,0,0,NaN
4,1,2016-03-01 05:39:54,0,0,NaN
...,...,...,...,...,...
23873,21492,2017-02-12 19:15:09,45,1,Fuite de consommation
23874,21492,2017-02-12 20:15:09,45,1,Fuite de consommation
23875,21492,2017-02-12 21:15:09,108,1,Fuite de consommation
23876,21492,2017-02-12 22:15:09,67,1,Fuite de consommation
